In [41]:
import requests
from io import BytesIO
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
def download_pdf(url: str):
    response = requests.get(url)
    return BytesIO(response.content)


downloaded_book = download_pdf(
    "https://ia800707.us.archive.org/31/items/milk-and-honey-by-rupi-kaur/milk-and-honey-by-rupi-kaur.pdf"
)

In [40]:
from pypdf import PdfReader


def parse_pages():
    reader = PdfReader(downloaded_book)
    return [page.extract_text() for page in reader.pages]


poems = parse_pages()
print(len(poems))

204
